# Домашнее задание № 4. Языковые модели

## Задание 1 (8 баллов).

В семинаре для генерации мы использовали предположение маркова и считали, что слово зависит только от 1 предыдущего слова. Но ничто нам не мешает попробовать увеличить размер окна и учитывать два или даже три прошлых слова. Для них мы еще сможем собрать достаточно статистик и, логично предположить, что качество сгенерированного текста должно вырасти.

Попробуйте сделать языковую модель, которая будет учитывать два предыдущих слова при генерации текста.
Сгенерируйте несколько текстов (3-5) и расчитайте перплексию получившейся модели. 
Можно использовать данные из семинара или любые другие (сопоставимые или большие по объему). Перплексию рассчитывайте на 10-50 отложенных предложениях (они не должны использоваться при сборе статистик).


Подсказки:  
    - нужно будет добавить еще один тэг <start>  
    - еще одна матрица не нужна, можно по строкам хронить биграмы, а по колонкам униграммы  
    - тексты должны быть очень похожи на нормальные (если у вас получается рандомная каша, вы что-то делаете не так). 

In [1]:
from string import punctuation
from razdel import sentenize
from razdel import tokenize as razdel_tokenize
import numpy as np
from IPython.display import Image
from IPython.core.display import HTML 

In [5]:
news = open('lenta.txt', encoding='utf-8').read()

In [6]:
def normalize(text):
    normalized_text = [word.text.strip(punctuation) for word \
                                                            in razdel_tokenize(text)]
    normalized_text = [word.lower() for word in normalized_text if word and len(word) < 20 ]
    return normalized_text

In [7]:
norm_news = normalize(news)

In [8]:
from collections import Counter

In [9]:
vocab_news = Counter(norm_news)

In [10]:
probas_news = Counter({word:c/len(norm_news) for word, c in vocab_news.items()})
probas_news.most_common(20)

[('в', 0.04808907489694771),
 ('и', 0.0221080111489724),
 ('на', 0.018883123731146926),
 ('по', 0.012943380513471676),
 ('что', 0.011310349590812525),
 ('с', 0.01057319451795703),
 ('не', 0.008435444806676101),
 ('из', 0.005131529052211166),
 ('о', 0.00499073907433246),
 ('как', 0.0049900749706632205),
 ('к', 0.00407161959610543),
 ('за', 0.0040125143695431435),
 ('россии', 0.0036751497055696383),
 ('для', 0.003325831175549828),
 ('его', 0.003260084912295149),
 ('он', 0.0031704309169478593),
 ('от', 0.003066830744546547),
 ('сообщает', 0.003050228152815567),
 ('а', 0.0029180715226369697),
 ('также', 0.002716184007188258)]

In [14]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\katri\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


True

In [46]:
from nltk.tokenize import sent_tokenize
def ngrammer(tokens, n=2):
    ngrams = []
    for i in range(0,len(tokens)-n+1):
        ngrams.append(' '.join(tokens[i:i+n]))
    return ngrams

In [17]:
from tqdm import tqdm

In [18]:
sentences_news = [['<start>', '<start>'] + normalize(text) + ['<end>'] for text in tqdm(sent_tokenize(news))]

100%|██████████████████████████████████████████████████████████████████████████| 76344/76344 [00:29<00:00, 2590.07it/s]


In [84]:
test_news = sentences_news[-50:]
sentences_news = sentences_news[:-50]

In [85]:
unigrams_news = Counter()
bigrams_news = Counter()
trigrams_news = Counter()

for sentence in sentences_news:
    unigrams_news.update(sentence)
    bigrams_news.update(ngrammer(sentence))
    trigrams_news.update(ngrammer(sentence, n=3))

In [86]:
trigrams_news.most_common(10)

[('<start> <start> в', 7961),
 ('<start> <start> по', 6210),
 ('<start> <start> как', 3736),
 ('<start> <start> однако', 1693),
 ('<start> <start> на', 1642),
 ('<start> <start> об', 1618),
 ('<start> об этом', 1578),
 ('<start> <start> он', 1551),
 ('<start> по словам', 1549),
 ('сообщает риа новости', 1324)]

In [39]:
from scipy.sparse import csr_matrix, csc_matrix, lil_matrix

In [87]:
matrix_news = lil_matrix((len(bigrams_news), 
                   len(unigrams_news)))

id2word_news = list(unigrams_news)
word2id_news = {word:i for i, word in enumerate(id2word_news)}

id2biword_news = list(bigrams_news)
biword2id_news = {word:i for i, word in enumerate(id2biword_news)}


for ngram in trigrams_news:
    word1, word2, word3 = ngram.split()
    bigram = word1 + ' ' + word2
    matrix_news[biword2id_news[bigram], word2id_news[word3]] =  (trigrams_news[ngram]/
                                                                     bigrams_news[bigram])

In [88]:
matrix_news = csr_matrix(matrix_news)

In [98]:
def generate(matrix, id2word, word2id, id2biword, biword2id, n=100, start='<start> <start>'):
    text = []
    current_idx = biword2id[start]
    
    for i in range(n):
        
        chosen = np.random.choice(list(range(matrix.shape[1])),
                                  p=matrix[current_idx].toarray()[0])
        text.append(id2word[chosen])
        
        prev_bigram = id2biword[current_idx].split()[1] + ' ' + id2word[chosen]
        current_idx = biword2id[prev_bigram]
        
        if id2word[chosen] == '<end>':
            current_idx = biword2id[start]
        
    return ' '.join(text)

##### Сгенерированные тексты

In [99]:
print(generate(matrix_news, id2word_news, word2id_news, id2biword_news, biword2id_news).replace('<end>', '\n'))

это уже не отрицают но и среди тех кому отказала а также закупать американское оружие и оборудование раздувая собственную значимость и на лестничных площадках выбиты окна и двери 
 как сообщили риа новости молодой человек претендовавший на руку интернетовским сми которые входят в правящее афганское движение талибан открыто встало на сторону бандитов 
 индивидуальные разрешения предоставляются на 5 процентов суммы вкладов превышающий 250-кратный размер минимального размера оплаты труда за каждого незаконно работающего иностранца в лес на 50-й километр ярославского шоссе столкнулись пять машин 
 в распространенном в среду государственная дума могла считаться 50 совладельцем сервера и получала возможность в самое


In [100]:
print(generate(matrix_news, id2word_news, word2id_news, id2biword_news, biword2id_news).replace('<end>', '\n'))

руководство футбольного клуба rad 
 по словам сотрудников правоохранительных органов 
 в результате столкновений с частями и подразделениями огв используя фальшивые документы силандии не заканчивается 
 главные претензии относились на орт а затем будет рассмотрен возможно уже в 2000 году планирует выдать 24-25 тысяч государственных жилищных сертификата для обеспечения общественного порядка уделяя особое внимание будет комплексу множество объектов которогорасположены на территории ставропольского края и области и татарии 
 хотя лидерство халонен выявилось в самом центре лондона где скапливаются огромные толпы народа 
 пожар начался в среду 
 между тем фсб россии по чечне не в меньшем а в интернете 



In [101]:
print(generate(matrix_news, id2word_news, word2id_news, id2biword_news, biword2id_news).replace('<end>', '\n'))

здание пиццерии уничтожено практически полностью контролируют обстановку в городе за минувшие сутки в развалинах дома 
 в 4 часа утра на два дня а покупать уголь и мазут затраты энергосистемы выросли на рекордную величину в 4,25 миллиона баррелей в день парламентских выборов в госдуму парламент хорватии в справедливости решения международного суда сомневаются 
 диагноз пока не вижу но внимание таким ситуациям сэр кажется вас надо арестовать за неуплату за два неудавшихся теракта с сухогруза кристина потерпевшего аварию 4 сентября 1998 года аппаратом владимира булгака занимавшего в то время как цена билета тут же взломали подменив главную страницу 
 он отметит


##### Подсчёт перплексии

Для оригинальных текстов из корпуса

In [102]:
def perplexity(probas):
    p = np.exp(np.sum(probas))
    N = len(probas)
    
    return p**(-1/N) 

In [145]:
def count_perplexity(sentences):
    all_probs = []
    for sent in sentences:
        probs = []
        for ngram in ngrammer(sent, n=3):
            word1, word2, word3 = ngram.split()
            bigram = word1 + ' ' + word2

            if bigram in bigrams_news and ngram in trigrams_news:
                probs.append(np.log(trigrams_news[ngram]/bigrams_news[bigram]))
            else:
                probs.append(np.log(0.00001))
        all_probs.append(probs)
    return np.mean([perplexity(p) for p in all_probs])

In [146]:
count_perplexity(test_news)

20004.95447677816

## Задание № 2* (2 балла). 

Прочитайте главу про языковое моделирование в книге Журафски и Мартина - https://web.stanford.edu/~jurafsky/slp3/3.pdf

Развернуто (в пределах 1000 знаков) ответьте на вопросы (по-русски):

1. Что можно делать с проблемой несловарных слов? В семинаре мы просто использовали какое-то маленькое значение вероятности, а какие есть другие способы?

В книге рассказано про ещё два способа, в каждом из которых реализована идея использования тэга UNK на разных стадиях:

1. Это можно делать in advance: задать заранее словарь слов, затем на этапе обучения все слова из обучающей выборки, которые туда не входят, заменять на тэг UNK и воспринимать эти тэги наравне с другими обычными словами из выборки.

2. Или можно уже на этапе обучения без предварительного словаря заменять слова с низкой частотностью (вручную задать границу N, ниже которой слова считаются неизвестными, или задать максимальное кол-во слов в словаре - топ-N по частотности как бы, а остальные признать неизвестными) на тэг UNK. Воспринимаются они так же наравне с другими словами.

2. Что такое сглаживание (smoothing)?

В тестовой выборке могут встретиться такие слова, которые были в словаре, но не встречались в обучающей выборке, или которые были в обучающей выборке, но, например, в другом контексте. Этим словам мы не можем присвоить тэг UNK, но присваивать им нулевую вероятность мы тоже не хотим. В таких случаях предлагается забирать немного вероятности от наиболее вероятных слов/контекстов и давать её этим словам/контекстам, которые не встречались в трейне, таким образом распределяя вероятность между переменными (наиболее вероятные всё равно останутся наиболее вероятными, зато этим невиданным переменным не будет проставляться 0).